In [1]:
#pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install streamlit-ace

Note: you may need to restart the kernel to use updated packages.


In [1]:
#pip install openai


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/947.6 kB ? eta -:--:--
   --------------------------------------- 947.6/947.6 kB 14.6 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import streamlit as st
from streamlit_ace import st_ace
import time
import tracemalloc
import os
import difflib

# ─────────────────────────────────────────────────────────────────────────────
# ✅ OpenAI 클라이언트 준비
# ─────────────────────────────────────────────────────────────────────────────
try:
    from openai import OpenAI
except ImportError:
    st.warning("openai 패키지가 필요합니다. 터미널에서 `pip install openai` 후 다시 실행하세요.")
    OpenAI = None

def get_client():
    api_key = None
    if "OPENAI_API_KEY" in st.secrets:
        api_key = st.secrets["OPENAI_API_KEY"]
    elif os.getenv("OPENAI_API_KEY"):
        api_key = os.getenv("OPENAI_API_KEY")
    if not api_key or not OpenAI:
        return None
    return OpenAI(api_key=api_key)

# ─────────────────────────────────────────────────────────────────────────────
# ⏱️ 실행 + 메모리 측정
# ─────────────────────────────────────────────────────────────────────────────
def run_code(code: str):
    tracemalloc.start()
    start_time = time.time()
    try:
        exec_globals = {}
        exec(code, exec_globals)  # 주의: 사용자 코드 직접 실행
        output = "코드 실행 완료"
    except Exception as e:
        output = f"오류 발생: {e}"
    runtime = time.time() - start_time
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return output, runtime, peak / 1024  # KB

# ─────────────────────────────────────────────────────────────────────────────
# 🌱 그린 알고리즘 프롬프트 (코드만)
# ─────────────────────────────────────────────────────────────────────────────
SYSTEM_PROMPT_CODE_ONLY = (
    "You are an expert Python performance engineer and green software specialist. "
    "Your task is to transform user-provided Python code into a functionally equivalent version "
    "that minimizes runtime and memory footprint. Apply algorithmic improvements first "
    "(lower time complexity, early exit, pruning, caching/memoization, vectorization, efficient data structures), "
    "then micro-optimizations (in-place ops, reduced allocations, streaming I/O, batching, avoiding global state). "
    "Return ONLY the final Python code. Do NOT include explanations, comments, markdown fences, or any text besides code."
)

def build_user_prompt(original_code: str, runtime_sec: float | None, memory_kb: float | None):
    rt = f"{runtime_sec:.6f}" if runtime_sec is not None else "unknown"
    mem = f"{memory_kb:.2f}" if memory_kb is not None else "unknown"
    return f"""
[GOAL]
- Reduce runtime and memory usage while preserving functionality and I/O behaviors.
- Keep the code self-contained and runnable as-is (no placeholders or external files).
- Use standard Python libraries only unless strictly necessary.

[CURRENT_METRICS]
- runtime_seconds: {rt}
- peak_memory_kb: {mem}

[CONSTRAINTS]
- Maintain same inputs/outputs and side effects unless they are provably redundant.
- Avoid unnecessary dependencies.
- Prefer clear, maintainable code over opaque micro-optimizations.

[FORMAT]
- Output the optimized Python code only.
- No backticks.
- No explanations.
- No comments.

[ORIGINAL_CODE]
{original_code}
""".strip()

def request_green_optimized_code(client: "OpenAI", model: str, original_code: str, runtime_sec: float | None, memory_kb: float | None) -> str:
    prompt = build_user_prompt(original_code, runtime_sec, memory_kb)
    resp = client.chat.completions.create(
        model=model,
        temperature=0.1,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_CODE_ONLY},
            {"role": "user", "content": prompt},
        ],
    )
    text = resp.choices[0].message.content or ""
    # 백틱 코드펜스 제거
    if text.strip().startswith("```"):
        text = text.strip().strip("`")
        if "\n" in text:
            first_line, rest = text.split("\n", 1)
            if first_line.strip().lower() in ("python",):
                text = rest
    return text.strip()

# ─────────────────────────────────────────────────────────────────────────────
# ✍️ 변경 이유/영향 요약용 프롬프트 (텍스트 설명)
# ─────────────────────────────────────────────────────────────────────────────
SYSTEM_PROMPT_EXPLAIN = (
    "You are a concise performance reviewer. Given ORIGINAL and OPTIMIZED Python code and metrics, "
    "summarize WHAT changed and WHY it improves runtime/memory. Be specific but brief."
)
def request_change_explanation(client: "OpenAI", model: str,
                               original_code: str, optimized_code: str,
                               base_rt: float | None, base_mem: float | None,
                               new_rt: float | None, new_mem: float | None) -> str:
    mr = lambda x: ("unknown" if x is None else (f"{x:.6f}" if isinstance(x, float) else str(x)))
    user_prompt = f"""
[CONTEXT]
- Original runtime (s): {mr(base_rt)}, Original peak memory (KB): {mr(base_mem)}
- Optimized runtime (s): {mr(new_rt)}, Optimized peak memory (KB): {mr(new_mem)}

[REQUEST]
- Provide a short bullet list of concrete changes (e.g., replaced recursion with iteration, added memoization, vectorized using built-ins, removed redundant copies).
- For each change, add a brief reason/impact (how it reduces time/memory).
- Keep it under ~10 bullets. Korean output.
- No code blocks.

[ORIGINAL]
{original_code}

[OPTIMIZED]
{optimized_code}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        temperature=0.2,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_EXPLAIN},
            {"role": "user", "content": user_prompt},
        ],
    )
    return (resp.choices[0].message.content or "").strip()

# ─────────────────────────────────────────────────────────────────────────────
# 🔍 Diff 생성
# ─────────────────────────────────────────────────────────────────────────────
def make_unified_diff(original_code: str, optimized_code: str) -> str:
    diff_lines = difflib.unified_diff(
        original_code.splitlines(),
        optimized_code.splitlines(),
        fromfile="original.py",
        tofile="optimized.py",
        lineterm=""
    )
    return "\n".join(diff_lines)

# ─────────────────────────────────────────────────────────────────────────────
# 🖥️ Streamlit UI
# ─────────────────────────────────────────────────────────────────────────────
st.title("Python 코드 실행기 (Ace Editor 포함)")

code = st_ace(
    language="python",
    theme="monokai",
    keybinding="vscode",
    font_size=14,
    height=300,
    auto_update=True,
)

# 원본 코드 실행
if st.button("코드 실행"):
    output, runtime, memory = run_code(code or "")
    st.write("### 실행 결과")
    st.write(output)
    st.write(f"### 실행 시간: {runtime:.4f} 초")
    st.write(f"### 메모리 사용량: {memory:.2f} KB")
    st.session_state["last_runtime"] = runtime
    st.session_state["last_memory"] = memory

st.divider()

# 🌿 최적화 + 자동 실행 + 변경 이유 표시
st.subheader("그린 알고리즘: 최적화 코드 추천 → 자동 실행 → 변경 요약")

default_model = "gpt-4.1-mini"
model_name = st.text_input("모델명", value=default_model, help="원하는 OpenAI 모델명을 입력하세요.")

if st.button("최적화 코드 생성 및 자동 실행 (코드만 반환)"):
    if not code or not code.strip():
        st.warning("먼저 상단 편집기에 코드를 입력하세요.")
    else:
        client = get_client()
        if client is None:
            st.error("OpenAI 클라이언트를 초기화하지 못했습니다. API 키 설정을 확인하세요.")
        else:
            base_rt = st.session_state.get("last_runtime")
            base_mem = st.session_state.get("last_memory")

            with st.spinner("그린 알고리즘 최적화 코드를 생성 중..."):
                try:
                    optimized = request_green_optimized_code(
                        client=client,
                        model=model_name.strip() or default_model,
                        original_code=code,
                        runtime_sec=base_rt,
                        memory_kb=base_mem,
                    )
                except Exception as e:
                    st.error(f"최적화 요청 중 오류가 발생했다: {e}")
                    optimized = None

            if optimized:
                st.write("#### 추천 코드 (GPT가 반환한 **코드만**)")
                st.code(optimized, language="python")

                # ➡️ 곧바로 실행
                out2, rt2, mem2 = run_code(optimized)
                st.write("#### 추천 코드 실행 결과")
                st.write(out2)
                st.write(f"실행 시간: {rt2:.4f} 초")
                st.write(f"메모리 사용량: {mem2:.2f} KB")

                # 비교
                if base_rt is not None and base_mem is not None:
                    drt = rt2 - base_rt
                    dmem = mem2 - base_mem
                    st.info(f"비교(추천 − 원본): Δ시간 {drt:+.4f} 초, Δ메모리 {dmem:+.2f} KB")

                # 🔄 Diff
                st.write("#### 변경된 코드 Diff")
                diff_text = make_unified_diff(code or "", optimized)
                if diff_text.strip():
                    st.code(diff_text, language="diff")
                else:
                    st.write("코드 변경점이 감지되지 않았습니다.")

                # 🗒️ 변경 이유/영향 (텍스트 설명)
                st.write("#### 변경 요약 및 이유")
                try:
                    explanation = request_change_explanation(
                        client=client,
                        model=model_name.strip() or default_model,
                        original_code=code or "",
                        optimized_code=optimized,
                        base_rt=base_rt, base_mem=base_mem,
                        new_rt=rt2, new_mem=mem2
                    )
                    st.markdown(explanation)
                except Exception as e:
                    # 클라이언트 불가/에러 시 최소한 Diff만 제공
                    st.warning(f"변경 이유 요약 생성 중 문제 발생: {e}\nDiff는 위에서 확인 가능합니다.")

                # 다운로드
                st.download_button(
                    label="최적화 코드 다운로드",
                    data=optimized.encode("utf-8"),
                    file_name="optimized.py",
                    mime="text/x-python",
                )


2025-09-16 09:59:42.637 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 09:59:44.159 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-16 09:59:44.160 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 09:59:44.160 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 09:59:44.161 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 09:59:44.161 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 09:59:44.162 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 09:59:44.162 Session state does not 